In [80]:
import pandas as pd
import requests
import json

In [50]:
dia_ontem = pd.Timestamp.today() -pd.Timedelta("1 days")
dia_ontem = dia_ontem.strftime('%Y-%m-%d')
hoje = pd.Timestamp.today().strftime('%Y-%m-%d')
dia_ontem

'2024-06-28'

In [102]:
# URL base da API
base_url = "https://dadosabertos.camara.leg.br/api/v2"

# Endpoint para obter as proposições
endpoint = "/proposicoes"

# Parâmetros da requisição
params = {
    "keywords": "Energia,Solar",  # Substitua por suas palavras-chave
    "dataInicio": '2024-06-28',
    "ano": [2024],
    "dataFim": '2024-06-29',      # Data de fim para filtrar proposições recentes
    "ordenarPor": "id"            # Ordenar por ID
}

# Cabeçalhos da requisição
headers = {
    "Accept": "application/json"
}

# Fazendo a requisição GET
response = requests.get("https://dadosabertos.camara.leg.br/api/v2/proposicoes?keywords=Solar&ordem=ASC&ordenarPor=id")
projetos_lista = []
# Verificando o status da resposta
if response.status_code == 200:
    # Convertendo a resposta para JSON
    dados = response.json()
    proposicoes = dados['dados']
    for proposicao in proposicoes:
        url_procura_id = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{proposicao['id']}"
        projeto_id = requests.get(url_procura_id)
        resposta_projeto = projeto_id.json()
        projeto_dict = {
            "ID": resposta_projeto['dados']['id'],
            "Tipo": resposta_projeto['dados']['siglaTipo'],
            "Ano": resposta_projeto['dados']['ano'],
            "Ementa": resposta_projeto['dados']['ementa'],
            "Data Inicio": resposta_projeto['dados']['dataApresentacao'],
            "Ultma Alteracao": resposta_projeto['dados']['statusProposicao']['dataHora'],
            "Palavras Chaves": resposta_projeto['dados']['keywords'],
            "Link": f"https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao={resposta_projeto['dados']['id']}"
        }
        projetos_lista.append(projeto_dict)
        
else:
    print(f"Erro na requisição: {response.status_code}")
relatorio_df = pd.DataFrame(projetos_lista)
relatorio_df.to_csv("Projetos-Novos.csv", index=False)